In [1]:
import os
import pandas as pd

import pickle
import random
import re

import kss
import ftfy

from collections import Counter

from tqdm.notebook import tqdm

In [2]:
data_path = 'C://Users/LGCNS/Documents/GitHub/Q_Bert/dt_v1/wiki_kor_210520_pages-articles/'

In [3]:
# 데이터 크기가 매우 크고, Local Disk 환경이 너무 작아 데이터를 업로드하는데 문제가 많습니다.
# 이에 따라 Opensource를 활용하여 Wiki Unzip(Extract)하겠습니다.

# https://github.com/attardi/wikiextractor

# !pip install wikiextractor

# python -m wikiextractor.WikiExtractor kowiki-20210520-pages-articles.xml

# Windows에서 WikiExtractor는 버그가 있어 사용하지 못합니다. Linux 응용프로그램을 설치하여 데이터를 준비했습니다.
# Warning: problems have been reported on Windows due to poor support for StringIO in the Python implementation on Windows.

In [4]:
# Recursively Get File Path
# 파일 경로만 확인하여 가져옵니다.

def get_recursive_files(file_path) :
    
    result_lst = []
    
    file_list = os.listdir(file_path)
    
    for f in file_list :
        
        target = os.path.join(file_path, f)
        
        if os.path.isdir(target) :
            
            result_lst += get_recursive_files(target)
        
        else :
            
            result_lst.append(target)
    
    return result_lst
    

In [5]:
full_path_lst = get_recursive_files(os.path.join(data_path, 'text'))

In [6]:
print(len(full_path_lst)) # 총 738개 파일

793


In [7]:
# 데이터 한 개를 불러와 데이터를 확인합니다.

test_file = full_path_lst[1]

with open(test_file, 'r', encoding = 'utf8') as f:
    txts = f.read()

In [8]:
import re

def clean_html(raw_html):
    
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def line_split(line) :
    
    all_lines = line.split('\n')
    
    filtered = list(filter(lambda x: x.strip()!='', all_lines))
    
    return filtered


In [9]:
def prep_txt(file_path) :
    
#     print(file_path, '\tSTART')
    
    global txt_dict
    
    #0. 불러오기
    with open(file_path, 'r', encoding = 'utf8') as f:
        txts = f.read()
    
    #1. <tag 제거>
    txt_offtag = clean_html(txts)

    #2. <\n\n\n\n> 토큰 단위 split
    txt_offtag_split_documents = txt_offtag.split('\n\n\n\n')

    #3. <\n> Filtering, 그리고 Dict화
    txt_split = list(map(line_split, txt_offtag_split_documents))

    for txt_s in txt_split :
        
        if len(txt_s) > 1 :
            txt_dict[txt_s[0]]= txt_s[1:]
#     print(file_path, "\tEND")
    return True

In [90]:
# 파일크기가 크므로 반으로 나누어 저장하도록 한다.

half_len = len(full_path_lst) // 2

In [91]:
txt_dict = {}

# 각각의 파일은 여러개의 Page가 포함된 Documents이다.
# 줄바꿈표시, <doc> </doc>과 같은 태깅이 포함되어있다.

# \n\n\n\n Token 별로 Documents를 형성하는 것으로 보임
# 소제목과 중제목 등이 포함되어있음. Min_Seq_Len를 이용한 Filtering

for path_one in full_path_lst[:half_len] :
    prep_txt(path_one)

with open(os.path.join(data_path, 'wiki_ko_dict1.pkl'), 'wb') as f:
    pickle.dump(txt_dict, f)

In [22]:
txt_dict = {}

for path_one in full_path_lst[half_len:] :
    prep_txt(path_one)

with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'wb') as f:
    pickle.dump(txt_dict, f)

## Sentence Split

In [77]:
# next Sentence 분석을 위해 문장 단위로 Split하여야한다.
# 작성자에 따라 문장이 Split되어 있거나 여러 문장이 한 문장으로 표현되어 있거나 한다.
# 소제목 등이 존재한다.
# 종종 utf8 기준 encoding이 깨져있는 경우도 존재한다.

# KSS Library를 이용하여 Split

In [13]:
def wikitext_detokenizer(string):
    # contractions
    string = string.replace("s '", "s'")
    string = re.sub(r"/' [0-9]/", r"/'[0-9]/", string)
    # number separators
    string = string.replace(" @-@ ", "-")
    string = string.replace(" @,@ ", ",")
    string = string.replace(" @.@ ", ".")
    # punctuation
    string = string.replace(" : ", ": ")
    string = string.replace(" ; ", "; ")
    string = string.replace(" . ", ". ")
    string = string.replace(" ! ", "! ")
    string = string.replace(" ? ", "? ")
    string = string.replace(" , ", ", ")
    # double brackets
    string = re.sub(r"\(\s*([^\)]*?)\s*\)", r"(\1)", string)
    string = re.sub(r"\[\s*([^\]]*?)\s*\]", r"[\1]", string)
    string = re.sub(r"{\s*([^}]*?)\s*}", r"{\1}", string)
    string = re.sub(r"\"\s*([^\"]*?)\s*\"", r'"\1"', string)
    string = re.sub(r"'\s*([^']*?)\s*'", r"'\1'", string)
    # miscellaneous
    string = string.replace("= = = =", "====")
    string = string.replace("= = =", "===")
    string = string.replace("= =", "==")
    string = string.replace(" " + chr(176) + " ", chr(176))
    string = string.replace(" \n", "\n")
    string = string.replace("\n ", "\n")
    string = string.replace(" N ", " 1 ")
    string = string.replace(" 's", "'s")
    
    # customize
    
    # doesn't include full/half transform
    string = string.replace("⁓", "~")
    # nothing except punctuation between parenthesis 
    string = re.sub(r"\s*\(\s*[., ]*\s*\)\s*", "", string)
    
    string = string.replace("(", " (")
    string = string.replace(")", ") ")

    return string

In [14]:
# Ver 2.

# 제외되는 문서 
# 1. 태그가 남은 문서

# 전처리
# 1. 전각문자 -> 반각문자
# 2. 영어, 한국어, 반각문자 특수문자 제외 삭제
# 3. 소문자 처리
# 4. 공백 문자 처리 (encoding + contiguous)
# 5. kss sentence_split
# 6. 양쪽 공백 제거, 빈 문장 제거
# 7. 문장 최소 길이 미만 문장 제거

def clean_text(sent) :

    clean_text = ftfy.fix_text(sent, normalization='NFKC')
    clean_text = re.sub("[^a-zA-Z0-9-ㄱ-ㅣ가-힣\-\+~=.,\&%@\\$#\?!\"\'\(\)\[\]\{\}\\s]", "", clean_text)
    set_clean = wikitext_detokenizer(clean_text)
    
    return set_clean

def transfer_sentences(lst, min_seq_len = 2) :
    
    total_txt = ''.join(lst)
    
    if (';' in total_txt) | ('colspan' in total_txt) :
        return []
    
    total_txt = clean_text(total_txt)
    
    total_txt = total_txt.lower()
    total_txt = total_txt.replace('\xa0', ' ')
    total_txt = total_txt.replace('  ', ' ')

    total_txts = kss.split_sentences(total_txt)
    
    total_txts = list(map(lambda x : x.strip(), total_txts))
    total_txts = list(filter(lambda x : (x != '') & (len(x.split(' ')) >= min_seq_len), total_txts))
    
    return total_txts

In [10]:
with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'rb') as f:
    txt_dict_pre = pickle.load(f)

In [16]:
save_data_path = data_path.replace('dt_v1', 'dt').replace('/wiki_kor_210520_pages-articles', '')

with open(os.path.join(data_path, 'wiki_ko_dict1.pkl'), 'rb') as f:
    txt_dict_pre = pickle.load(f)

key = list(txt_dict_pre.keys())
txt_dict_post = {}    
for key in tqdm(list(txt_dict_pre.keys())) :
    tmp_txt = transfer_sentences(txt_dict_pre[key], 3)
    # 문장이 한 개 이상인 Documents만 사용
    if len(tmp_txt) > 1 :
        txt_dict_post[key] = tmp_txt

print(len(list(txt_dict_post.keys())))
        
with open(os.path.join(save_data_path, 'wiki_ko_dict1_p.pkl'), 'wb') as f:
    pickle.dump(txt_dict_post, f)

  0%|          | 0/176610 [00:00<?, ?it/s]

134085


FileNotFoundError: [Errno 2] No such file or directory: 'C://Users/LGCNS/Documents/GitHub/Q_Bert/dt/wiki_kor_210520_pages-articles/wiki_ko_dict1_p.pkl'

In [19]:
with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'rb') as f:
    txt_dict_pre = pickle.load(f)

print(len(list(txt_dict_pre.keys())))
    
txt_dict_post = {}    
for key in tqdm(txt_dict_pre.keys()) :
    tmp_txt = transfer_sentences(txt_dict_pre[key], 3)
    # 문장이 한 개 이상인 Documents만 사용
    if len(tmp_txt) > 1 :
        txt_dict_post[key] = tmp_txt

print(len(list(txt_dict_post.keys())))
        
with open(os.path.join(save_data_path, 'wiki_ko_dict2_p.pkl'), 'wb') as f:
    pickle.dump(txt_dict_post, f)

356970


  0%|          | 0/356970 [00:00<?, ?it/s]

221621


## Sample 100

In [24]:
data_path_old = data_path
data_path = save_data_path

In [21]:
with open(os.path.join(data_path, 'wiki_ko_dict1_p.pkl'), 'rb') as f:
    txt_dict_post = pickle.load(f)

In [22]:
import numpy as np


key_lst = list(txt_dict_post.keys())
total_len = len(key_lst)



r_indexs = np.random.randint(0, total_len, 100)

In [23]:
for r_index in r_indexs :
    
    key = key_lst[r_index]
    
    print("===================================================")
    print(key)
    print(txt_dict_post[key])

쿠아우테목
['쿠아우테목 (cuauhtmoc, 1502년 - 1525년 2월 28일) 또는 과티모신 (guatimozin) 은 아스텍 제국의 제11대 황제로 마지막 황제이다.', '그는 몬테수마 2세의 친척이자 사위였다.', '"쿠아우테목"이라는 이름의 의미는 "독수리 같은 후손"이라는 뜻의 나우아틀어로, 영어에서 "추락하는 독수리" (falling eagle) 로 해석하는 것은 오류이다.', '몬테수마 2세와의 정확한 관계는 알려지지 않았으나 보통, 악사카야틀의 아들이자 몬테수마 2세의 형제 (이름 미상) 의 아들이라는 설과 몬테수마 2세의 삼촌인 8대 황제 아우이트소틀의 손자라는 설이 있다.', '그는 친척인 목테수마 2세의 딸로 도나 이사벨 목테수마 (doa isabel moctezuma) 로 알려진 테추키포 (tecuichpo) 와 결혼하였다.', '쿠아우테목은 1520년 몬테수마의 후계자 쿠이틀라우악이 천연두로 죽은 뒤 황제가 되었다.', '그러나 그는 18세의 어린 전사였고, 일부 귀족들은 황제로 인정하지 않거나 정통성에 의문을 제기하였다.', '그 무렵 막강한 인디오 내부의 동맹세력을 가지고 있던 에르난 코르테스가 아스텍의 수도인 테노치티틀란으로 진군해오고 있었다.', '1521년 쿠아우테목의 국경 수비군은 후퇴하지 않을 수 없었다.', '쿠아우테목은 포위공격에 맞서 4개월간 수도를 방어했으나 도시의 대부분이 파괴되고 인디오들은 거의 전멸했다.', '스페인군에 붙잡힌 그는 처음에는 예우를 받았으나, 나중에는 아스텍의 보물이 숨겨진 곳을 대라는 고문을 당했다.', '모진 고문에도 불구하고 끝내 입을 열지 않은 그의 극기는 전설이 되었다.', '쿠아우테목을 남겨두고 떠날 경우 후환을 우려한 코르테스는 그를 온두라스로 데리고 가던 중 스페인을 겨냥한 음모가 있다는 소식을 듣고 쿠아우테목을 교수형에 처하도록 지시했다.', '1949년 멕시코 익스카테오판에서 그의 것으로 보이는 유골이 발굴되었다.', '그가 전사한 뒤 아즈텍은 스페인의 직할 식민지가 되었고

## Sentence로 저장

In [25]:
import os
import pickle
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# data_path = 'C://Users/LGCNS/Documents/GitHub/Q_Bert/dt/wiki_kor_210520_pages-articles/'

In [26]:
with open(os.path.join(data_path, 'wiki_ko_dict1_p.pkl'), 'rb') as f:
    dict1 = pickle.load(f)

with open(os.path.join(data_path, 'wiki_ko_dict2_p.pkl'), 'rb') as f:
    dict2 = pickle.load(f)

In [27]:
total_sentence_num = 0
unique_word = set()

for sentences in tqdm_notebook(list(dict1.values())) :
    
    for sentence in sentences :
        
        total_sentence_num+= 1
#         unique_word = set(list(unique_word) + sentence.split(' '))
        
        with open('./dt/train_wiki_sentence1.txt', 'a', encoding = 'utf8') as f:

            f.write(sentence+'\n')


  0%|          | 0/134085 [00:00<?, ?it/s]

In [28]:
print(total_sentence_num)
# print(len(unique_word))

1807314


In [29]:
for sentences in tqdm_notebook(list(dict2.values())) :
    
    for sentence in sentences :
        
        total_sentence_num+= 1
#         unique_word = set(list(unique_word) + sentence.split(' '))
        
        with open('./dt/train_wiki_sentence2.txt', 'a', encoding = 'utf8') as f:

            f.write(sentence+'\n')


  0%|          | 0/221621 [00:00<?, ?it/s]

In [30]:
print(total_sentence_num)
# print(len(unique_word))

3506777


In [ ]:

with open('./dt/wiki_ko_dict1_p.pkl', 'rb') as f:
    df = pickle.load(f)
    
keys = list(df.keys())
r_key = random.choices(keys,k=100)

smp_dict = {}

for key in r_key :
    smp_dict[key] = df[key]
    
with open('./dt/wiki_ko_dict_sample.pkl', 'wb') as f:
    pickle.dump(smp_dict, f)